<a href="https://colab.research.google.com/github/NavdeepTU/Deep-Learning/blob/main/02.%20ML%20basics%20with%20Keras/03.%20Exercise_Multi_class_classification_on_Stack_Overflow_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Predict the tag of a programming question on Stack Overflow

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

In [9]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset = tf.keras.utils.get_file('stack_overflow_16k', url, untar = True, cache_dir='.', cache_subdir='')
dataset_dir = os.path.dirname(dataset)

In [10]:
os.listdir(dataset_dir)

['.config',
 'stack_overflow_16k.tar.gz',
 'test',
 'train',
 'README.md',
 'sample_data']

In [11]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['java', 'javascript', 'csharp', 'python']

In [12]:
sample_file = os.path.join(train_dir, 'python/30.txt')
with open(sample_file) as f:
  print(f.read())

"blank script trigger a method in another script i have two scripts say script1.py on pc1 and script2.py on pc2. now, script1.py has two methods as shown below. the method1() on pc1 must be completed before method2() on pc2 can be executed. so, how can i trigger method2() of script2.py on pc2 which is already running?..i can split script2.py into two different .py files i.e method1.py and method2.py on pc2 and call method2.py after script1_method2() on pc1 has finished but then i need to pass the foo object to it. maybe i can pickle the foo object and read it from a file?..is there a better way to go about this?..note: i can communicate between pc1 and pc2 using paramiko ssh.. def script1_method1():.    """""".    this is on pc1.    get connection from pc2 script 2 and do some work.    """""".    pass..def script1_method2():.    """""".    this is on pc1.    work which needs some time.    only after this work is done, call pc2 script2.    """""".    pass..def script2_method1():.    """

Load the dataset

In [13]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
    'train',
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'training',
    seed = seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [16]:
for text_batch, label_batch in raw_train_ds.take(1):  # taking only 1 batch
  for i in range(3):
    print('Question : ', text_batch.numpy()[i])
    print('Label : ', label_batch.numpy()[i])

Question :  b'"unit testing of setters and getters teacher wanted us to do a comprehensive unit test. for me, this will be the first time that i use junit. i am confused about testing set and get methods. do you think should i test them? if the answer is yes; is this code enough for testing?..  public void testsetandget(){.    int a = 10;.    class firstclass = new class();.    firstclass.setvalue(10);.    int value = firstclass.getvalue();.    assert.asserttrue(""error"", value==a);.  }...in my code, i think if there is an error, we can\'t know that the error is deriving because of setter or getter."\n'
Label :  1
Question :  b'"static class, static constructors and static properties i have a static class that has only static properties and a static constructor. when i try to access or set the value of property (with a backing field) the static constructor is not called. however, if i define a static method and try to call it the constructor is executed...i believe properties are just

In [17]:
print('Label 0 corresponds to ', raw_train_ds.class_names[0])
print('Label 1 corresponds to ', raw_train_ds.class_names[1])
print('Label 2 corresponds to ', raw_train_ds.class_names[2])
print('Label 3 corresponds to ', raw_train_ds.class_names[3])

Label 0 corresponds to  csharp
Label 1 corresponds to  java
Label 2 corresponds to  javascript
Label 3 corresponds to  python


In [18]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    'train',
    batch_size = batch_size,
    validation_split = 0.2,
    subset = 'validation',
    seed = seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [19]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    'test',
    batch_size = batch_size
)

Found 8000 files belonging to 4 classes.
